In [9]:
import ast
import scipy
import numpy as np
import pandas as pd

'''
lets use a real modeling tool....

import pysgpp
from pysgpp.extensions.datadriven.learner import LearnerBuilder
from pysgpp.extensions.datadriven.uq.plot.plotGrid import plotGrid
from pysgpp.extensions.datadriven.uq.plot.plot1d import plotSG1d
from pysgpp.extensions.datadriven.uq.plot.plot2d import plotSG2d
'''

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
filename = 'output_100_configs.csv'

df = pd.read_csv(filename, header = None, names = ['params', 'time'])
df.params = df.params.apply(ast.literal_eval)

for param in df.iloc[0].params.keys():
    df[param] = df.params.apply(lambda row: row[param])
    
df['density'] = (df.MOL_X*df.MOL_Y)/(df.DOMAIN_SIZE_X*df.DOMAIN_SIZE_Y)
del df['params']
df = df[df.time > 0]

In [5]:
df.head(15)

,time,DOMAIN_SIZE_X,TEMP,DOMAIN_SIZE_Y,DELTA_T,V_x,V_y,LINKED_CELL_SIZE_Y,LINKED_CELL_SIZE_X,DOMAIN_SIZE,MOL_X,MOL_Y,CUTOFF_RADIUS,TIMESTEPS,BLOCK_SIZE,CHECKPOINT,density
0,1446.798254,43,2.5,44,0.002,1,1.5,31,31,5,42,40,2.00,1000,1,checkpoints/P0VMMLFY83_999,0.887949
1,527.306368,44,2.5,41,0.002,1,1.5,36,21,5,40,41,4.50,1000,1,checkpoints/EUW7N08GQD_999,0.909091
2,77.665511,40,2.5,40,0.002,1,1.5,26,21,5,42,41,3.75,1000,10,checkpoints/TX65RKKAM3_999,1.076250
3,237.649865,44,2.5,42,0.002,1,1.5,36,11,5,42,44,4.75,1000,1,checkpoints/8DFSGNILRV_999,1.000000
4,19.883890,44,2.5,43,0.002,1,1.5,16,11,5,42,42,1.50,1000,1,checkpoints/A54W7Y2SCV_999,0.932347
5,45.849872,40,2.5,42,0.002,1,1.5,16,26,5,41,43,4.25,1000,1,checkpoints/MCGP05L7Q8_999,1.049405
6,113.095191,40,2.5,40,0.002,1,1.5,11,36,5,42,41,3.25,1000,10,checkpoints/PYQRTFSES9_999,1.076250
7,1763.922973,42,2.5,43,0.002,1,1.5,36,36,5,42,44,3.50,1000,1,checkpoints/PREOIN4N55_999,1.023256
8,26.505233,40,2.5,42,0.002,1,1.5,11,21,5,40,40,1.25,1000,1,checkpoints/CAP5QOY6WB_999,0.952381
9,145.585394,43,2.5,44,0.002,1,1.5,16,31,5,42,42,2.75,1000,1,checkpoints/FBG9RH9F9S_999,0.932347


In [6]:
params = ['DOMAIN_SIZE_X', 'DOMAIN_SIZE_Y', 'MOL_X', 'MOL_Y', 'LINKED_CELL_SIZE_X', 'LINKED_CELL_SIZE_Y',
          'BLOCK_SIZE', 'CUTOFF_RADIUS', 'density']

In [7]:
num_samples = df.shape[0]

X = df[params].values
y = df.time.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [35]:
svms = [svm.SVR(kernel='rbf', C=1e3, gamma=0.1), svm.SVR(kernel='linear', C=1e3),
        svm.SVR(kernel='poly', C=1e3, degree=2)]
map(lambda svm: svm.fit(X_train, y_train), svms)

[SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
   kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 SVR(C=1000.0, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
   kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)]

In [37]:
print map(lambda svm: svm.score(X_train, y_train), svms)
print map(lambda svm: svm.score(X_test, y_test), svms)

[0.9950261465810093, 0.55964516515816043, 0.80313629244746321]
[0.73817469792025336, 0.56739064771505887, 0.84083403830558923]


polynomial svm with $d=2$ already reaches 85% on $R^2$ of residules of validation set. Lets investigate other parameters

In [ ]:
params = {'C': scipy.stats.expon(scale=100), 'gamma': scipy.stats.expon(scale=.1),
          'kernel': ['rbf', 'poly'], 'degree':range(1,4)}

random_search = RandomizedSearchCV(svm.SVR(), param_distributions = params, n_iter = 20)
random_search.fit(X_train, y_train)
print random_search.score(X_test, y_test)

In [ ]:
f = pysgpp.createOperationEval(learner.grid)
print "f(x) = {}".format(f.eval(learner.alpha, X))